## This notebook demonstrates how to propagate an orbit back in time, then compute a state transition matrix on the forward propagation

In [1]:
import sys
from os.path import expanduser

# default adam location - alter directory structure if you decide to use a different path
# for example: adam_home_defined = expanduser("~") + "your_project_name/scenario1/adam_home"
adam_home_defined = expanduser("~") + "/adam_home" # default location

# Add adam path to system in order to import adam
sys.path.append(adam_home_defined)

# import adam and adam modules required
import adam
from adam import adam_config

In [2]:
from adam import Batch
from adam import PropagationParams
from adam import OpmParams
from adam import BatchRunManager
from adam import ConfigManager
from adam import Service
from adam import StmPropagationModule

import os
import datetime
import numpy as np
import numpy.testing as npt
import pandas as pd

In [3]:
# Set up adam paths
ADAMpaths = adam_config.setPaths.initPaths(adam_home_defined)

Changing adam home path to =  /Users/macuser/adam_home


In [4]:
#Attempts to read config that is already set up from an adam_config.json file in adam_home/config directory.
#Feel free to point it somewhere else. If it fails to read that file, it will copy the template from adam.
config_file  = ADAMpaths[3]
template_file = ADAMpaths[2]

try:
    f = open(config_file)
    f.close()
except:
    copyfile(template_file, config_file)

This sets up a Service which uses the given token and URL to provide authorized access through the server through several wrapped modules. It also creates a project for you to work in that will be used for the rest of the notebook. Be sure to run service.teardown() when finished. 

If you don't have a config, see config_demo to get one.

In [5]:
# Reads your config from a file in current directory. For instructions on setting this up, see config_demo notebook.
# config = ConfigManager(os.getcwd() + '/config.json').get_config()
config_manager = ConfigManager(config_file)
config = config_manager.get_config()
service = Service(config)
service.setup()

[12.575406] Setup


True

In [6]:
working_project = service.new_working_project()

Set up project with uuid abe32b01-09b1-4763-80ee-75cda6f3b7c7
[2.541897] Generate working project


### Definitions

Following are definitions of all the parameters we would like to use, including regarding the asteroid, its location, and the times we want to use in our propagations.

In [7]:
asteroid_params = {
    'mass': 500.5,              # object mass
    'solar_rad_area': 25.2,     # object solar radiation area (m^2)
    'solar_rad_coeff': 1.2,     # object solar radiation coefficient
    'drag_area': 33.3,          # object drag area (m^2)
    'drag_coeff': 2.5,          # object drag coefficient
    'object_name': 'KillerAsteroid'
}

initial_state_vector = [1.9614856544727500000e7,
                        1.4843205548512800000e8,
                        5.5892860024181600000e7,
                        -28.4909386978969000000,
                        -5.3762767683990300000,
                        2.5848941723961300000]

propagation_time = 50 * 365 * 24 * 60 * 60  # 50 years in seconds.

now = datetime.datetime(2018, 2, 21, 0, 0, 0, 123456)
fifty_years_ago = now - datetime.timedelta(seconds=propagation_time)

### Backwards propagation

This creates and runs a propagation backwards 50 years from now.

In [8]:
backwards_propagation = Batch(PropagationParams({
    'start_time': now.isoformat() + 'Z',
    'end_time': fifty_years_ago.isoformat() + 'Z',
    'step_size': 0,  # Use adaptive step size.
    'project_uuid': working_project.get_uuid(),
}), OpmParams(dict(asteroid_params, **{
    'epoch': now.isoformat() + 'Z',
    'state_vector': initial_state_vector,
})))

batch_run_manager = BatchRunManager(service.get_batches_module(), [backwards_propagation])
batch_run_manager.run();

# Should be 'COMPLETED'.
print('Finished backwards propagation in state %s' % (backwards_propagation.get_calc_state()))
state_vector_fifty_years_ago = backwards_propagation.get_results().get_end_state_vector()
print('State vector fifty years ago: %s' % state_vector_fifty_years_ago)

[0.980043] Submitting 1 runs.
[12.989589] Running.
[1.734956] Retrieving propagation results.
Finished backwards propagation in state COMPLETED
State vector fifty years ago: [-105079893.82459801, 55761751.36629267, 39539057.70658225, -14.973064259000001, -31.308238753, -9.959259069]


TODO(laura): This is actually not true because python notebooks do not run cells simultaneously. There are ways to make this work. Do it.

In case you're curious, you can run the following to get the state of your runs (in this case only backwards_propagation) while running.

In [9]:
status = batch_run_manager.get_latest_statuses()
status_counts = [[s, len(status[s])] for s in status]
df = pd.DataFrame(status_counts, columns=['Task', 'Status'])
df.style.hide_index()

Task,Status
PENDING,0
RUNNING,0
COMPLETED,1
FAILED,0


### STM calculation

Now that we know where this asteroid was 50 years ago, let's propagate forward using the STM module to compute a state transition matrix that we can use in our delta-V calculations.

In [10]:
forwards_propagation_params = PropagationParams({
    'start_time': fifty_years_ago.isoformat() + 'Z',
    'end_time': now.isoformat() + 'Z',
    'step_size': 0,  # Use adaptive step size.
    'project_uuid': working_project.get_uuid(),
})
forwards_opm_params = OpmParams(dict(asteroid_params, **{
    'epoch': fifty_years_ago.isoformat() + 'Z',
    'state_vector': state_vector_fifty_years_ago,  # This will be nudged to compute the STM.
}))

stm_propagator = StmPropagationModule(service.get_batches_module())
now_state_vector, stm = stm_propagator.run_stm_propagation(forwards_propagation_params, forwards_opm_params)

# We should have ended up quite close to the same position and velocity at which it started. Validate that.
difference = np.subtract(now_state_vector, initial_state_vector)
npt.assert_allclose(difference[0:3], [0, 0, 0], rtol=0, atol=1e-3)    # Allow 1m of change in position
npt.assert_allclose(difference[3:6], [0, 0, 0], rtol=0, atol=1e-10)   # But only a tiny change in velocity

# Now we have the state transition matrix!
print("State transition matrix:")
print(stm)

[0.516483] Submitting 13 runs.
[59.404287] Running.
[16.410745] Retrieving propagation results.


AssertionError: 
Not equal to tolerance rtol=0, atol=0.001

(mismatch 100.0%)
 x: array([-2641186.193074,  -643221.722065,   180860.782719])
 y: array([0, 0, 0])

### Now what?

Now that we have the state transition matrix, how do we compute delta-Vs? What should happen now? TODO(hackathon)

In [11]:
# Cleans up working project and any batches and runs that were made within that project.
service.teardown()

Cleaning up working project abe32b01-09b1-4763-80ee-75cda6f3b7c7...
[1.46866] Teardown
